In [2]:
!pip install ipywidgets plotly
!jupyter nbextension enable --py widgetsnbextension

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 37.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [ipywidgets]3 [ipywidgets]

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python3 -m pip install --upgrade pip
usage: jupyter [-h] [--version] [--config-dir] [--data-dir] [--runtime-dir]
               [--paths] [--json] [--debug]
               [subcommand]

Jupyter: Interactive Computing

positional arguments:
  subcommand     the subcommand to launch

options:
  -h, --help     show this help message and exit
  --version      show the versions of core jupyter packages and exit
  --config-dir   show Jupyter config dir
  --data-dir     show Jupyter data dir
  --runtime-dir  show Jupyter runtime dir
  --paths        show all Jupyter paths. Add --json for machine-readable
                 format.
  --json         output paths as machine-readable json
  --debug        output debug information about paths

Available subcommands: 

In [ ]:
import sqlite3
import pandas as pd
import plotly.express as px
import ipywidgets as widgets
from pathlib import Path
from IPython.display import display

# Path to SQLite database
DB_PATH = Path("../data/health.db")  # adjust if notebook is not in repo root

def get_connection():
    return sqlite3.connect(DB_PATH)

# --- Load filter options ---
with get_connection() as con:
    states = sorted(pd.read_sql_query("SELECT DISTINCT state FROM patients ORDER BY state;", con)["state"])
    conditions = sorted(pd.read_sql_query("SELECT DISTINCT condition_name FROM conditions ORDER BY condition_name;", con)["condition_name"])

# --- Widgets ---
state_dropdown = widgets.Dropdown(
    options=["All"] + states,
    value="All",
    description="State:",
    layout=widgets.Layout(width="250px")
)

condition_dropdown = widgets.Dropdown(
    options=["All"] + conditions,
    value="All",
    description="Condition:",
    layout=widgets.Layout(width="400px")
)

# --- Chart functions ---
def plot_readmit_rate(state_filter, condition_filter):
    with get_connection() as con:
        query = """
        WITH filtered_encounters AS (
            SELECT e.*
            FROM encounters e
            JOIN patients p ON e.patient_id = p.patient_id
            {where_clause}
        )
        SELECT date(strftime('%Y-%m-01', start_date)) AS month_start,
               AVG(CASE WHEN was_readmit='True' THEN 1.0 ELSE 0.0 END) AS readmit_rate
        FROM filtered_encounters
        GROUP BY month_start
        ORDER BY month_start;
        """

        # Build WHERE clause
        clauses = []
        params = {}
        if state_filter != "All":
            clauses.append("p.state = :state")
            params["state"] = state_filter
        if condition_filter != "All":
            query = """
            WITH cond_patients AS (
                SELECT DISTINCT patient_id
                FROM conditions
                WHERE condition_name = :cond
            ),
            filtered_encounters AS (
                SELECT e.*
                FROM encounters e
                JOIN patients p ON e.patient_id = p.patient_id
                JOIN cond_patients cp ON e.patient_id = cp.patient_id
                {where_clause}
            )
            SELECT date(strftime('%Y-%m-01', start_date)) AS month_start,
                   AVG(CASE WHEN was_readmit='True' THEN 1.0 ELSE 0.0 END) AS readmit_rate
            FROM filtered_encounters
            GROUP BY month_start
            ORDER BY month_start;
            """
            params["cond"] = condition_filter

        where_clause = ""
        if clauses:
            where_clause = "WHERE " + " AND ".join(clauses)
        query = query.format(where_clause=where_clause)

        df = pd.read_sql_query(query, con, params=params)

    if df.empty:
        return px.line(title="No data for selected filters")

    fig = px.line(df, x="month_start", y="readmit_rate",
                  title="Monthly 30-day Readmission Rate",
                  markers=True)
    fig.update_layout(yaxis_tickformat=".0%")
    return fig

def plot_top_conditions(state_filter):
    with get_connection() as con:
        query = """
        SELECT condition_name, COUNT(DISTINCT c.patient_id) AS patients
        FROM conditions c
        JOIN patients p ON c.patient_id = p.patient_id
        {where_clause}
        GROUP BY condition_name
        ORDER BY patients DESC
        LIMIT 10;
        """
        params = {}
        where_clause = ""
        if state_filter != "All":
            where_clause = "WHERE p.state = :state"
            params["state"] = state_filter

        query = query.format(where_clause=where_clause)
        df = pd.read_sql_query(query, con, params=params)

    if df.empty:
        return px.bar(title="No data for selected filters")

    fig = px.bar(df, x="patients", y="condition_name", orientation="h",
                 title="Top Conditions by Patient Count")
    fig.update_layout(yaxis=dict(autorange="reversed"))
    return fig

# --- Interactive dashboard ---
def update_dashboard(state_filter, condition_filter):
    fig1 = plot_readmit_rate(state_filter, condition_filter)
    fig2 = plot_top_conditions(state_filter)
    fig1.show()
    fig2.show()

widgets.interactive(update_dashboard,
                    state_filter=state_dropdown,
                    condition_filter=condition_dropdown)

